# nbs_to_ctx

> Turning Jupyter notebooks into LLM-ready context

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|export
import os, glob, nbformat
from xml.sax.saxutils import escape
from fastcore.script import call_parse, Param

In [ ]:
#| export
def find_ipynb_files(directory):
    """Find all .ipynb files in the directory and its subdirectories."""
    return glob.glob(os.path.join(directory, '**', '*.ipynb'), recursive=True)

In [ ]:
find_ipynb_files('.')

['./index.ipynb', './00_core.ipynb']

In [ ]:
#| export
def render_notebook_to_markdown(nb_path):
    """Convert a Jupyter notebook to markdown, including markdown cells, code cells, and limited text outputs."""
    # Read the notebook file
    nb = nbformat.read(nb_path, as_version=4)
    # Get the programming language from metadata, default to 'python'
    language = nb.metadata.get('kernelspec', {}).get('language', 'python')
    content = []

    # Process each cell in the notebook
    for cell in nb.cells:
        if cell.cell_type == 'markdown':
            # Include markdown cell content as is
            content.append(''.join(cell.source))
        elif cell.cell_type == 'code':
            # Include code cell content in a markdown code block
            code = ''.join(cell.source)
            content.append(f"```{language}\n{code}\n```")
            # Process text-based outputs, ignoring large or non-text outputs
            outputs = []
            for output in cell.outputs:
                if output.output_type == 'stream':
                    # Stream outputs (e.g., stdout/stderr)
                    text = ''.join(output.text)
                    outputs.append(text[:200] + (' ... (truncated)' if len(text) > 200 else ''))
                elif output.output_type in ['execute_result', 'display_data']:
                    # Results or displayed data with plain text
                    if 'text/plain' in output.data:
                        text = ''.join(output.data['text/plain'])
                        outputs.append(text[:200] + (' ... (truncated)' if len(text) > 200 else ''))
            if outputs:
                content.append("**Outputs:**\n" + '\n'.join(outputs))
    
    # Join cell contents with double newlines for separation
    return '\n\n'.join(content)

In [ ]:
render_notebook_to_markdown('./index.ipynb')[:50]

'```python\n#| hide\nfrom nbs2ctx.core import *\n```\n\n'

In [ ]:
#| export
def generate_xml(directory):
    """Generate an XML string containing markdown-rendered content from all notebooks."""
    ipynb_files = sorted(find_ipynb_files(directory))
    documents = []
    
    # Process each notebook file
    for index, file in enumerate(ipynb_files, start=1):
        rendered = render_notebook_to_markdown(file)
        # Use relative path from the input directory as the source
        source = os.path.relpath(file, directory)
        # Escape content for XML compatibility
        escaped_content = escape(rendered)
        # Build the document XML structure
        documents.append(
            f'<document index="{index}">\n'
            f'<source>{source}</source>\n'
            f'<document_content>\n{escaped_content}\n</document_content>\n'
            f'</document>'
        )
    
    # Wrap all documents in the root <documents> tag
    return '<documents>\n' + '\n'.join(documents) + '\n</documents>'

In [ ]:
generate_xml('.')[:150]

'<documents>\n<document index="1">\n<source>00_core.ipynb</source>\n<document_content>\n# nbs_to_ctx\n\n&gt; Turning Jupyter notebooks into LLM-ready context'

In [ ]:
#|export
@call_parse
def main(
    directory: Param(help='Directory containing .ipynb files'),
    output:   Param(help='Output XML file')
):
    "Convert .ipynb files to XML for LLMs"
    xml_content = generate_xml(directory)
    with open(output, 'w', encoding='utf-8') as f:
        f.write(xml_content)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()